In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,2449
2,Huelva,Confirmados PDIA 14 días,751
3,Huelva,Tasa PDIA 14 días,"147,04068386644556"
4,Huelva,Confirmados PDIA 7 días,400
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,1
629,Municipio de Huelva sin especificar,Total Confirmados,34
630,Municipio de Huelva sin especificar,Curados,6


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  2449.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  643.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1512 personas en los últimos 7 días 

Un positivo PCR cada 617 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,2449.0,400.0,751.0,510743.0,78.317275,147.040684,51.0
Huelva-Costa,1272.0,212.0,424.0,288115.0,73.581730,147.163459,25.0
Sierra de Huelva-Andévalo Central,338.0,76.0,112.0,67571.0,112.474286,165.751580,19.0
Huelva (capital),643.0,95.0,233.0,143663.0,66.126978,162.185114,15.0
Condado-Campiña,805.0,111.0,206.0,155057.0,71.586578,132.854370,14.0
Cortegana,49.0,34.0,39.0,4666.0,728.675525,835.833691,10.0
Lepe,191.0,44.0,67.0,27431.0,160.402464,244.249207,5.0
Almonte,95.0,28.0,38.0,24191.0,115.745525,157.083213,5.0
Palma del Condado (La),118.0,17.0,33.0,10761.0,157.977883,306.662950,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),26.0,14.0,24.0,516.0,2713.178295,4651.162791,0.0
Cortegana,49.0,34.0,39.0,4666.0,728.675525,835.833691,10.0
Villarrasa,31.0,4.0,18.0,2176.0,183.823529,827.205882,1.0
Cerro de Andévalo (El),21.0,14.0,16.0,2364.0,592.216582,676.818951,3.0
Marines (Los),2.0,2.0,2.0,396.0,505.050505,505.050505,0.0
Villanueva de los Castillejos,26.0,7.0,14.0,2820.0,248.226950,496.453901,1.0
Valdelarco,3.0,1.0,1.0,239.0,418.410042,418.410042,0.0
Linares de la Sierra,1.0,1.0,1.0,263.0,380.228137,380.228137,NaN
Jabugo,16.0,2.0,8.0,2250.0,88.888889,355.555556,1.0
